# Note :


```
I am using the same dataset which i used in other unsupervised Deep learning 

algorithm Boltzmann machine hence Data preprocessing steps are same
```



In [ ]:
"""
Importing the libraries 

"""

import numpy as np

import pandas as pd

import torch

import torch.nn as nn

import torch.nn.parallel

import torch.optim as optim

import torch.utils.data

from torch.autograd import Variable

In [ ]:
"""

importing the Dataset

"""

movies = pd.read_csv("/content/drive/My Drive/SelfPractise/Boltzmann Machine/ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv("/content/drive/My Drive/SelfPractise/Boltzmann Machine/ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv("/content/drive/My Drive/SelfPractise/Boltzmann Machine/ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

movies.head()


# Training set

In [ ]:

""" 
Preparing Training and Test Set 
NOte : Separater for this file is Tab hence using \t parameter
I am converting these dataset into numpy array because I am using PyTorch Tensors
"""

training_set = pd.read_csv("/content/drive/My Drive/SelfPractise/Boltzmann Machine/ml-100k/u1.base", delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv("/content/drive/My Drive/SelfPractise/Boltzmann Machine/ml-100k/u1.test", delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [ ]:
"""

Getting the total number of users and Movies, Why Am I doing this?

Because I will be converting my training_Set and test_Set into a matrix where lines are going to user and columns are 
going to be movies and cells are going to be the ratings.

I will create such matrix for training set and test set both

I will also include older users, older movies from original dataset into these two Matrixes 

And The Training set that we just imported, If a user did not rate the movies, We will put it a zero
into the cell of matrix that corresponds to  this user and that movie

"""

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [ ]:
"""

Going to convert our dataset into an array with users in lines and movies in columns

Why Am i doing this?

We will need to make a specific structure that will corresponds to  what the Restricted Boltzmann machine wants as input

These observations will go one by one into the RBM. Hence will create a structure that will contain these observations
that will go into the Network and their different features that are going to be into input nodes and that is what
i will do by creating this array with users in lines and movies in the columns because we will have observations in lines and 
features in the columns. 

In other words, It is the same like we do in all neural networks or ML ALgorithms.

I will accomplish this objective by creating a user defined function. I will give only one argument to this function
because it will apply to only one set of data which will training set first and when we will apply this to our training set 

or test set, We only need to replace this argument by training set and test set, Then colon :

after than I will tell the function what it needs to do


"""


def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:

""" Converting the data into torch tensor """

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Architecture of the Autoencoders

In [ ]:
""" 

This will be stacked auto encoder hence neurons of hidden layers are stacked 

First Fully connected layer will accept the input vector and number of neurons

second fully connected layer will accept the number of nuerons from previous hidden layer and number of neurons we want to

assign at this layer

I am not making this architecture very dense. But Check out my experiment repo. where I have performed so many experiments

with dense autoencoder architectures and lightweight also.


"""

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training the Autoencoder: 

SAE = Stacked Autoencoder

In [ ]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

# Testing the Autoencoder:

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))